In [1]:
import numpy as np
import pandas as pd
import lightfm
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

In [2]:
from utils import evaluate, load_data

read data

In [3]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

##### split data

In [4]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [5]:
u_train = ui_train.user_id
i_train = ui_train.item_id
likes_train = ui_train.like
dislikes_train = ui_train.dislike

u_val = ui_val.user_id
i_val = ui_val.item_id
likes_val = ui_val.like
dislikes_val = ui_val.dislike

In [6]:
sparse_train = coo_matrix((likes_train-dislikes_train, (u_train, i_train)))
sparse_val = coo_matrix((likes_val-dislikes_val, (u_val, i_val)))

user_features = user_meta_data[["gender", "age"]].values
item_features = item_meta_data[["source_id", "duration"]].values
user_features_train = csr_matrix(user_features)
item_features_train = csr_matrix(item_features)

##### lightfm model

In [10]:
model = lightfm.LightFM(no_components=32)

In [49]:
model.fit(sparse_train,
          epochs=5,
          num_threads=8,
          verbose=True)

Epoch: 100%|██████████| 5/5 [02:37<00:00, 31.55s/it]


In [50]:
val_lightfm_score = model.predict(u_val.values, i_val.values)

In [51]:
ui_val["target"] = ui_val["like"] - ui_val["dislike"]
ui_val["lightfm_score"] = val_lightfm_score

In [52]:
evaluate(user_id=ui_val.user_id.values,
         target=ui_val.target.values,
         score=ui_val.lightfm_score.values)

100%|██████████| 181721/181721 [00:52<00:00, 3467.31it/s]


np.float64(0.6178085542103173)

##### hyperopt

In [7]:
from hyperopt import hp, fmin, tpe, Trials

In [8]:
space = {
    'no_components': hp.uniformint('no_components', 8, 128),
    'learning_schedule': hp.choice('learning_schedule', ["adagrad", "adadelta"]),
    'loss': hp.choice('loss', ["bpr", "warp", "warp-kos"]),
    "num_epochs": hp.uniformint("num_epochs", 5, 50),
}

In [9]:
def objective(params):
    num_epochs = params.pop("num_epochs")
    model = lightfm.LightFM(**params)
    model.fit(sparse_train,
              epochs=num_epochs,
              num_threads=8,
              verbose=True)
    val_lightfm_score = model.predict(u_val.values, i_val.values)
    ui_val["target"] = ui_val["like"] - ui_val["dislike"]
    ui_val["lightfm_score"] = val_lightfm_score
    score = evaluate(user_id=ui_val.user_id.values,
                     target=ui_val.target.values,
                     score=ui_val.lightfm_score.values)
    return -score

In [10]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=25)

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

100%|##########| 181721/181721 [00:53<00:00, 3427.02it/s]


  4%|▍         | 1/25 [10:25<4:10:18, 625.79s/trial, best loss: -0.6113211844375209]

100%|##########| 181721/181721 [00:53<00:00, 3409.55it/s]


  8%|▊         | 2/25 [18:10<3:23:31, 530.92s/trial, best loss: -0.6113211844375209]

...

 92%|█████████▏| 23/25 [6:51:45<1:24:38, 2539.23s/trial, best loss: -0.6123845424510608]

100%|##########| 181721/181721 [00:52<00:00, 3494.40it/s]


 96%|█████████▌| 24/25 [7:05:43<33:48, 2028.98s/trial, best loss: -0.6124258034780476]  

100%|##########| 181721/181721 [00:52<00:00, 3494.36it/s]


100%|██████████| 25/25 [7:19:43<00:00, 1055.36s/trial, best loss: -0.6124258034780476]


In [11]:
best

{'learning_schedule': np.int64(0),
 'loss': np.int64(0),
 'no_components': np.float64(111.0),
 'num_epochs': np.float64(50.0)}

##### submission

In [98]:
test_lightfm_score = model.predict(test_pairs_data.user_id.values,
                                   test_pairs_data.item_id.values,
                                   None)

In [100]:
test_pairs_data["predict"] = test_lightfm_score
test_pairs_data.to_csv("./submission.csv",index=False)